RNN

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 샘플 텍스트 데이터
data = """나는 오늘 기분이 좋아.
나는 내일도 기분이 좋을 거야.
기분이 좋은 날엔 춤을 추고 싶어."""

In [2]:
# 1. 토큰화: 텍스트 데이터를 숫자로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.split('\n'))
sequences = tokenizer.texts_to_sequences(data.split('\n'))

In [3]:
# 2. 단어 인덱스 확인
word_index = tokenizer.word_index
print("단어 인덱스:", word_index)

단어 인덱스: {'기분이': 1, '나는': 2, '오늘': 3, '좋아': 4, '내일도': 5, '좋을': 6, '거야': 7, '좋은': 8, '날엔': 9, '춤을': 10, '추고': 11, '싶어': 12}


In [4]:
# 3. 시퀀스를 학습 데이터로 변환
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        input_sequences.append(sequence[:i+1])

In [5]:
# 4. 패딩 처리
max_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, 
                                maxlen=max_len, 
                                padding='pre')

In [6]:
# 5. 입력(X)과 출력(y) 분리
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
print(X)
# 6. 출력(y)을 원-핫 인코딩
y = tf.keras.utils.to_categorical(y, num_classes=len(word_index) + 1)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  0  0  0  2]
 [ 0  0  0  2  5]
 [ 0  0  2  5  1]
 [ 0  2  5  1  6]
 [ 0  0  0  0  1]
 [ 0  0  0  1  8]
 [ 0  0  1  8  9]
 [ 0  1  8  9 10]
 [ 1  8  9 10 11]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [8]:
# RNN 모델 정의
rnn_model = Sequential([
    Embedding(input_dim=len(word_index) + 1, 
              output_dim=10, 
              input_length=max_len- 1), # 임베딩 층
              SimpleRNN(64, return_sequences=False), # RNN 층
              Dense(len(word_index) + 1, activation='softmax') # 출력 층
              ])

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# 모델 컴파일 및 학습
rnn_model.compile(loss='categorical_crossentropy',
                   optimizer='adam', 
                   metrics=['accuracy'])
rnn_model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 2.5564
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0833 - loss: 2.5435
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0833 - loss: 2.5307
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1667 - loss: 2.5179
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3333 - loss: 2.5049
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3333 - loss: 2.4915
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3333 - loss: 2.4775
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.3333 - loss: 2.4629
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3333 - loss: 2.4474
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3333 - loss: 2.4310
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3333 - loss: 2.4135
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3333 - loss: 2.3949

In [10]:
# LSTM 모델 정의
lstm_model= Sequential([
    Embedding(input_dim=len(word_index) +1, 
              output_dim=10, 
              input_length=max_len-1), # 임베딩 층
              LSTM(64, return_sequences=False), # LSTM 층
              Dense(len(word_index) +1,
                    activation='softmax') # 출력 층
                    ])
 # 모델 컴파일 및 학습
lstm_model.compile(loss='categorical_crossentropy', 
                   optimizer='adam', 
                   metrics=['accuracy'])
lstm_model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0833 - loss: 2.5645
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1667 - loss: 2.5624
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1667 - loss: 2.5603
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2500 - loss: 2.5582
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2500 - loss: 2.5560
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1667 - loss: 2.5538
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.1667 - loss: 2.5515
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1667 - loss: 2.5491
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1667 - loss: 2.5466
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1667 - loss: 2.5440
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1667 - loss: 2.5412
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1667 - loss: 2.5383
Epo

In [11]:
# GRU 모델 정의
gru_model= Sequential([
    Embedding(input_dim=len(word_index) +1,
              output_dim=10,
              input_length=max_len-1), # 임베딩 층
  GRU(64, return_sequences=False), # GRU 층
  Dense(len(word_index) +1, 
activation='softmax') # 출력 층
])
 # 모델 컴파일 및 학습
gru_model.compile(loss='categorical_crossentropy',
 optimizer='adam', 
metrics=['accuracy'])
gru_model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 2.5661
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0833 - loss: 2.5627
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.0833 - loss: 2.5593
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0833 - loss: 2.5559
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1667 - loss: 2.5525
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1667 - loss: 2.5490
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1667 - loss: 2.5454
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1667 - loss: 2.5418
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1667 - loss: 2.5380
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3333 - loss: 2.5340
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2500 - loss: 2.5299
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2500 - loss: 2.5256
Epoch 13/2

In [14]:
def generate_text(model, tokenizer, seed_text, next_words, max_len):
 for _ in range(next_words):
    token_list =tokenizer.texts_to_sequences([seed_text])[0]
    token_list =pad_sequences([token_list], maxlen=max_len-1, padding='pre')
    predicted =np.argmax(model.predict(token_list, verbose=0), axis=-1)
    for word, index in tokenizer.word_index.items():
        if index== predicted:
            seed_text += " " + word
            break
 return seed_text

In [15]:
# 텍스트 생성 예제
seed_text="나는 오늘"
print("RNN 생성 결과:", generate_text(rnn_model, tokenizer, seed_text, next_words=5, max_len=max_len))
print("LSTM 생성 결과:", generate_text(lstm_model, tokenizer, seed_text, next_words=5, max_len=max_len))
print("GRU 생성 결과:", generate_text(gru_model, tokenizer, seed_text, next_words=5, max_len=max_len))

RNN 생성 결과: 나는 오늘 기분이 기분이 기분이 기분이 기분이
LSTM 생성 결과: 나는 오늘 기분이 기분이 기분이 기분이 기분이
GRU 생성 결과: 나는 오늘 기분이 기분이 좋은 기분이 좋은
